In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
x = Lar.Struct([square])

LinearAlgebraicRepresentation.Struct([([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])], [[0.0; 0.0], [1.0; 1.0]], "16978137412824715025", 2, "feature")

## Vecchia funzione

In [4]:
function embedStruct(n::Int)
	function embedStruct0(self::Lar.Struct,suffix::String="New")
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = [ [self.body[i].box[1];zeros(dimadd)],
						[self.body[i].box[2];zeros(dimadd)] ]
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end

embedStruct (generic function with 1 method)

In [5]:
@btime embedStruct(1)(x)

LoadError: UndefVarError: i not defined

In [6]:
@code_llvm embedStruct(1)(x)


;  @ In[4]:2 within `embedStruct0'
; Function Attrs: uwtable
define nonnull %jl_value_t* @julia_embedStruct0_2090([1 x i64]* nocapture nonnull readonly dereferenceable(8), %jl_value_t* nonnull align 8 dereferenceable(40)) #0 {
top:
;  @ In[4]:3 within `embedStruct0'
  %2 = call nonnull %jl_value_t* @j_embedStruct0_2091([1 x i64]* nocapture nonnull readonly %0, %jl_value_t* nonnull %1, %jl_value_t* inttoptr (i64 353419792 to %jl_value_t*)) #0
  ret %jl_value_t* %2
}


In [7]:
@benchmark embedStruct(1)(x)

LoadError: UndefVarError: i not defined

## Modifiche

"""\
	embedStruct(n::Int)(self::Struct,suffix::String)::Struct\
Restituisce una struttura che è la copia della struttura “self” passata come
parametro, dove però la dimensione è aumentata del fattore “n” e al nome è
posto il suffisso. Per copiare il body è chiamata “embedTraversal”.\
"""

In [8]:
function embedStruct(n)
	function embedStruct0(self, suffix)
		if n==0
			return self, length(self.box[1])
		end
		cloned = Lar.Struct()
		cloned.box = hcat((self.box,[fill([0],n),fill([0],n)]))
		cloned.name = self.name*suffix
		cloned.category = self.category
		cloned.dim = self.dim+n
		cloned = Lar.embedTraversal(cloned,self,n,suffix)
		return cloned
	end
	return embedStruct0
end

embedStruct (generic function with 2 methods)

In [9]:
@btime embedStruct(1)(x,"-New")

LoadError: UndefVarError: i not defined

In [10]:
@benchmark embedStruct(1)(x,"-New")

LoadError: UndefVarError: i not defined

In [11]:
@code_typed embedStruct(1)(x,"-New")

CodeInfo(
1 ─ %1  = Core.getfield(#self#, :n)::Int64
│   %2  = (%1 === 0)::Bool
└──       goto #3 if not %2
2 ─ %4  = Base.getfield(self, :box)::Any
│   %5  = Base.getindex(%4, 1)::Any
│   %6  = Main.length(%5)::Any
│   %7  = Core.tuple(self, %6)::Tuple{LinearAlgebraicRepresentation.Struct,Any}
└──       return %7
3 ─ %9  = Base.getproperty(Main.Lar, :Struct)::Any
│   %10 = (%9)()::Any
│   %11 = Base.getfield(self, :body)::Array
│   %12 = Base.getindex(%11, Main.i)::Any
│   %13 = Base.getproperty(%12, :box)::Any
│   %14 = Base.getindex(%13, 1)::Any
│   %15 = Main.zeros(Main.dimadd)::Any
│   %16 = Base.vcat(%14, %15)::Any
│   %17 = Base.getfield(self, :body)::Array
│   %18 = Base.getindex(%17, Main.i)::Any
│   %19 = Base.getproperty(%18, :box)::Any
│   %20 = Base.getindex(%19, 2)::Any
│   %21 = Main.zeros(Main.dimadd)::Any
│   %22 = Base.vcat(%20, %21)::Any
│   %23 = Base.vect(%16, %22)::Array{_A,1} where _A
│         Base.setproperty!(%10, :box, %23)::Any
│   %25 = Base.getfield(self, 

In [12]:
@code_llvm embedStruct(1)(x,"-New")


;  @ In[4]:2 within `embedStruct0'
; Function Attrs: uwtable
define nonnull %jl_value_t* @julia_embedStruct0_2256([1 x i64]* nocapture nonnull readonly dereferenceable(8), %jl_value_t* nonnull align 8 dereferenceable(40), %jl_value_t* nonnull) #0 {
top:
  %3 = alloca %jl_value_t*, i32 4
  %gcframe = alloca %jl_value_t*, i32 6, align 16
  %4 = bitcast %jl_value_t** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* align 16 %4, i8 0, i32 48, i1 false)
  %5 = call %jl_value_t*** inttoptr (i64 1761836128 to %jl_value_t*** ()*)() #7
;  @ In[4]:3 within `embedStruct0'
  %6 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 0
  %7 = bitcast %jl_value_t** %6 to i64*
  store i64 16, i64* %7
  %8 = getelementptr %jl_value_t**, %jl_value_t*** %5, i32 0
  %9 = getelementptr %jl_value_t*, %jl_value_t** %gcframe, i32 1
  %10 = bitcast %jl_value_t** %9 to %jl_value_t***
  %11 = load %jl_value_t**, %jl_value_t*** %8
  store %jl_value_t** %11, %jl_value_t*** %10
  %12 = bitcast %jl_value_t*

In [13]:
@benchmark embedStruct(1)(x,"-New")

LoadError: UndefVarError: i not defined